In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import tensorflow as tf
import ast

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import word embeddings and reformat
import gensim
from gensim.models.keyedvectors import KeyedVectors
word_vectors = gensim.models.Word2Vec.load('nearby_embeddings2')
wordVectors = []
for word in word_vectors.wv.index2word:
    wordVectors.append(word_vectors.wv[word])
wordVectors = np.array(wordVectors)
wordsList = word_vectors.wv.index2word

max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [6]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [7]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [8]:
len(raw_data), len(y_list)

(16623, 16623)

In [9]:
word_tokenize(raw_data['body'].iloc[11]), raw_data['original_text'].iloc[11], y_list[11]

(['No', 'I', 'ask', 'Paytm', 'debit', 'card', 'queries'],
 ['paytm debit'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
ids = np.load('nearby_ids2.npy')

In [11]:
char_vocab = []
def get_character_set(text):
    for char in text:
        if not char in char_vocab:
            char_vocab.append(char)

raw_data['body'].apply(lambda x: get_character_set(x))

0        None
1        None
2        None
3        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
21287    None
21288    None
21289    None
21290    None
21291    None
21292    None
21293    None
21294    None
21295    None
21296    None
21297    None
21298    None
21299    None
21301    None
21302    None
21304    None
21305    None
21306    None
21307    None
21309    None
21310    None
21311    None
21312    None
21313    None
21316    None
21318    None
21319    None
21320    None
21321    None
21322    None
Name: body, Length: 16623, dtype: object

In [12]:
char_vocab_len = len(char_vocab)
char_one_hot = np.zeros([char_vocab_len, char_vocab_len])
for i in range(len(char_vocab)):
    char_one_hot[i][i] = 1
    

In [13]:
max_char_length = 16
max_sequence_length = 20
char_dimensions = char_vocab_len
batch_size = 100
raw_data_len = len(raw_data)
ids_char = np.zeros([(raw_data_len),max_sequence_length,max_char_length ])
ids_char = ids_char + len(char_vocab) -1

In [14]:
g = 0
for i in (range((raw_data_len))):
    body = raw_data.iloc[i]['body']
    words = word_tokenize(body)
    for j in (range(len(words))):
        word = words[j]
        for k in range(len(word)):
            char = word[k]
            try:
                ids_char[i][j][k] = char_vocab.index(char)
            except:
                pass

In [15]:
ids_char.shape

(16623, 20, 16)

In [16]:
import tensorflow as tf
tf.reset_default_graph()


In [17]:
#weights = {'W_conv1': tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_conv2':                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_fc':tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }



#biases = {'b_conv1':tf.get_variable("b_conv1", shape=([32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_conv2':tf.get_variable("b_conv2", shape=([64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_fc':tf.get_variable("b_fc", shape=([1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([(int(10//2))*(int(4//2))*32,100])),
           'out':tf.Variable(tf.random_normal([100, 10]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
           'b_conv2':tf.Variable(tf.random_normal([64])),
           'b_fc':tf.Variable(tf.random_normal([100])),
           'out':tf.Variable(tf.random_normal([10]))}
n_nodes_hl1 = 40
hidden_1_layer = {'weights':tf.Variable(tf.random_normal([max_char_length * char_dimensions, n_nodes_hl1])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}


def character_level_embedding(char_input_data):
    print('sdiugfh', char_input_data)
    char_input_data = tf.cast(char_input_data, dtype=tf.int32)
    char_one_hot_tensor = tf.convert_to_tensor(char_one_hot, dtype=tf.int32)
    char_data = tf.placeholder(shape = (max_sequence_length,max_char_length), dtype=tf.int32)
    char_data = tf.nn.embedding_lookup(char_one_hot_tensor,char_input_data)
    print('char_data', char_data)
    keep_rate = 1
    print('aya')
    
    #print(char_data.shape)
#The place holder to hold the values of new x and y at different times.
    x = tf.placeholder(dtype=tf.float32, shape=[max_sequence_length, max_char_length * char_dimensions])
    x = tf.cast(tf.reshape(char_data, shape=[max_sequence_length, max_char_length * char_dimensions]), dtype=tf.float32)
    #print(x)
#y = tf.placeholder('float', [batch_size, n_classes])

    def embedding_layer(data):
        print('aya')
        l1 = tf.add(tf.tensordot(data,hidden_1_layer['weights'], axes=1), hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)
        print(l1.shape)
        return l1

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

    def maxpool2d(x):
        #                        size of window         movement of window
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


    def convolutional_neural_network(x):
        
#        tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#        
#                        tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())

        
        

        print('og', x.shape)
        x = tf.reshape(x, shape=[max_sequence_length,10,4, 1])
        
        conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
        conv2 = maxpool2d(conv1)

        #conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
        #conv2 = maxpool2d(conv2)
        
        print('con2',conv2.shape)
        fc = tf.reshape(conv2,[max_sequence_length, (10//2) * (4//2) * 32])
        fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
        fc = tf.nn.dropout(fc, keep_rate)
        #print(fc.shape)
        return fc

    def train_neural_network(x):    
        embedded_x = embedding_layer(x)
        print(embedded_x)
        #embedded_x = tf.reshape(x, [max_sequence_length, char_dimensions])
        convoluted_x = convolutional_neural_network(embedded_x)
        
        return convoluted_x
    
    char_data = train_neural_network(x)
    return char_data


char_input_data = tf.placeholder(dtype=tf.float32,shape= [batch_size, max_sequence_length, max_char_length])
char_embeddings = tf.map_fn(lambda x: character_level_embedding(x), char_input_data, dtype=tf.float32)



sdiugfh Tensor("map/while/TensorArrayReadV3:0", shape=(20, 16), dtype=float32)
char_data Tensor("map/while/embedding_lookup:0", shape=(20, 16, 158), dtype=int32)
aya
aya
(20, 40)
Tensor("map/while/Relu:0", shape=(20, 40), dtype=float32)
og (20, 40)
con2 (20, 5, 2, 32)


In [18]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    p = ids_char.astype(int)[100:200]
    print(type(p[0][0][0]))
    b = sess.run([char_embeddings], feed_dict={char_input_data: p})
    print(b[0].shape)
    

<class 'numpy.int64'>
(100, 20, 100)


In [19]:
lstmUnits = 64
numClasses = 2
epochs = 10
numDimensions = 100
batchSize = 100
train_len = 14000

X_train = ids[:train_len]
y_train = y_list[:train_len]
X_test = ids[train_len:]
y_test = y_list[train_len:]
ids_char_train = ids_char[:train_len]
ids_char_test = ids_char[train_len:]




#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test, ids_char_train, ids_char_test = train_test_split(ids,y_list,ids_char, test_size=.2)

len(X_train), len(y_test), len(ids_char_train)

(14000, 2623, 14000)

In [20]:
def getTrainBatch(j):
    i = j*batchSize
    arr = X_train[i:i+batchSize]
    labels = y_train[i:i+batchSize]
    arr_char = ids_char_train[i:i+batchSize]
    #print(labels[0])
    return arr, labels, arr_char

In [21]:
def getTestBatch(j):
    i = j*batchSize
    arr = X_test[i:i+batchSize]
    labels =   y_test[i:i+batchSize]
    arr_char = ids_char_test[i:i+batchSize]
    return arr, labels, arr_char

In [22]:
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_sequence_length])

In [23]:
wordVectors = tf.convert_to_tensor(wordVectors, dtype=tf.float32)

In [24]:
data = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
print(data.shape)
print(char_embeddings.shape)
data2 = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions+char_dimensions), dtype=tf.float32)
data2 = tf.concat([data, char_embeddings], axis=2)

(100, 20, 100)
(100, 20, 100)


In [25]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [26]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data2,dtype=tf.float32)

In [27]:
#print(outputs)

outputs = tf.concat(outputs, 2)
#print(outputs.shape)

In [28]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [29]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
print(prediction.shape)
p2 = tf.argmax(labels, axis=2)
print('p2', p2.shape)
print(labels)

prediction = tf.cast(prediction, dtype=tf.float32)

p2 = tf.cast(p2, dtype=tf.int32)

sequence_lengths = tf.constant(shape=[batch_size], value=np.zeros(batch_size)+max_sequence_length, dtype=tf.int32)


log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
inputs=prediction, 
sequence_lengths=sequence_lengths,
tag_indices=p2)
crf_loss = tf.reduce_mean(-log_likelihood)

prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)

#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
#with tf.Session() as sess:
    #tf.initialize_all_variables().run()
#    sess.run(init_g)
#    sess.run(init_l)
#    nextBatch, nextBatchLabels, nextChars = getTrainBatch(2)
        #print(j/2)
#    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
#    print(pa.shape, la.shape, fp, fn, tp, tn)
    #a = pred[0]
    #b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))

correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


(100, 20, 2)
p2 (100, 20)
Tensor("Placeholder_1:0", shape=(100, 20, 2), dtype=float32)


In [30]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(crf_loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [31]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())#

#    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
#    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

In [32]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [33]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(X_train)//batchSize):
        nextBatch, nextBatchLabels, nextChars = getTrainBatch(j);
        #print(j/2)
        ll, _,pred,lab = sess.run([loss, optimizer,prediction,labels], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
        print(ll)
        losses += ll

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
print((np.array(pred)))
x3 = np.array(lab)
writer.close()

epoch:  0
3.2089145
0.58777016
0.25660294
0.31067193
0.30239204
0.2981282
0.24577542
0.18730216
0.1663501
0.19103147
0.3199651
0.36113778
0.31454897
0.16540821
0.16904415
0.1892319
0.17010663
0.17114301
0.18706717
0.16872644
0.18129542
0.1648379
0.18875621
0.14573795
0.1582699
0.18326807
0.17238195
0.15212142
0.11812002
0.11806517
0.11509068
0.12645595
0.35371923
0.2758272
0.18679877
0.13993624
0.11184164
0.107159846
0.09701684
0.104766466
0.12085079
0.14123453
0.16543515
0.16385582
0.16560449
0.14199229
0.12480528
0.12620436
0.1065555
0.09468372
0.16208658
saved to models/pretrained_lstm.ckpt-0
0.1739752
0.17652786
0.15531015
0.15494518
0.16044468
0.16215402
0.13522199
0.16824323
0.1425406
0.14528549
0.13091913
0.13865127
0.17269546
0.10686761
0.12121295
0.12704152
0.12238188
0.12206229
0.16025607
0.12647761
0.13302033
0.12892097
0.13069625
0.09182829
0.09228712
0.11561259
0.1024742
0.09962355
0.09974915
0.105128326
0.11197075
0.10678634
0.10189722
0.09371518
0.09160359
0.09299728
0.1

0.123156615
0.116934635
0.0937097
0.08962483
0.06670513
0.07091505
0.0751812
0.07472287
0.059902757
0.069198444
10.747586134821177
epoch:  5
0.078481995
0.08969721
0.05522634
0.058157418
0.06206103
0.06701637
0.05332175
0.06336172
0.07008361
0.054244142
0.04491777
0.06803549
0.07838321
0.09136994
0.07735588
0.07966327
0.07891577
0.077341646
0.0779147
0.0578003
0.080439836
0.060358237
0.056423727
0.048577763
0.055472948
0.05443831
0.06847005
0.06704316
0.060887642
0.06840409
0.06723602
0.059177157
0.084080674
0.08576904
0.085423276
0.08360677
0.05417227
0.061024353
0.060539093
0.056534797
0.058142524
0.053741623
0.06752722
0.06527719
0.07481433
0.0680456
0.060339637
0.07003256
0.07120891
0.051495004
0.09616737
saved to models/pretrained_lstm.ckpt-5
0.0907696
0.08128953
0.08141094
0.07941955
0.084247865
0.08607705
0.06631936
0.08374309
0.06542425
0.082981326
0.096823774
0.093854114
0.10114071
0.05440922
0.0570245
0.08122974
0.06822423
0.08706578
0.13480842
0.08527683
0.08728862
0.0616491

0.10338087
0.06242531
0.052372452
0.047387954
0.048469592
0.05882784
0.050701927
0.059022985
0.06222447
0.06554567
0.07344431
0.061898254
0.06342
0.07041483
0.0874097
0.07395242
0.08478896
0.049371183
0.08065788
0.045456536
0.053937733
0.056829784
0.0676781
0.10696481
0.100023754
0.08332951
0.09259599
0.057041015
0.06188948
0.063305326
0.06628085
0.05321753
0.06340561
9.641795996576548
[[[-1.5717528   2.237339  ]
  [ 1.7676694  -2.3187826 ]
  [ 2.1959803  -3.0519433 ]
  ...
  [ 4.209931   -4.2146807 ]
  [ 4.0712123  -4.0431666 ]
  [ 3.8175678  -3.9057288 ]]

 [[ 3.1329033  -1.4759827 ]
  [ 1.1233137   1.2426417 ]
  [ 1.2461579   0.37279734]
  ...
  [ 4.1043315  -4.488279  ]
  [ 3.9574552  -4.2845826 ]
  [ 3.7830553  -4.2031775 ]]

 [[ 1.8082453  -0.6393304 ]
  [ 1.3051951  -0.31536952]
  [-1.1310829   2.0274944 ]
  ...
  [ 4.186154   -4.2739987 ]
  [ 4.0542507  -4.099404  ]
  [ 3.8131595  -3.9659078 ]]

 ...

 [[ 1.0852603  -0.6548227 ]
  [ 0.20259528  0.71755576]
  [-2.4906206   2.683

In [34]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 1])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatch, nextBatchLabels, nextChars = getTestBatch(j);
    p, t = sess.run([prediction,transition_params], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
    
    for k in range((batchSize)):
        viterbi_sequence, viterbi_score = tf.contrib.crf.viterbi_decode(p[k], t)
        print(np.array(viterbi_sequence).shape)
        viterbi_sequence = np.array(viterbi_sequence).reshape(20,1)
        fin_pred[jj:jj+100][k] = viterbi_sequence

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,

In [35]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(fin_pred[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [36]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [37]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [38]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(3763, 683, 673, 0.8463787674313991, 0.8482867448151488, 0.8473316820535914)

In [39]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [40]:
prediction_analysis_pandas

,body,original_text,pred_original_text,false_positives,true_positives,false_negatives
0,"[i, call, on, just, dial, .., its, better, the...",[samsung],"[its, better, samsung]","{its, better}",{samsung},{}
1,"[i, want, ambala, bus, stand, context, no, .]","[bus, stand]","[ambala, bus, stand, context]","{ambala, context}","{stand, bus}",{}
2,"[Beauty, parlour, for, women]","[Beauty, parlour]","[Beauty, parlour]",{},"{parlour, Beauty}",{}
3,"[Its, not, a, night, club, dear]","[night, club]",[club],{},{club},{night}
4,"[More, Nearby, :, workshop]",[workshop],[workshop],{},{workshop},{}
5,"[Not, send, me, agai]",[agai],[send],{send},{},{agai}
6,"[Car, workshop]","[Car, workshop]","[Car, workshop]",{},"{Car, workshop}",{}
7,[Daraganj],[Daraganj],[Daraganj],{},{Daraganj},{}
8,"[Look, for, him]","[for, him]",[Look],{Look},{},"{for, him}"
9,"[Nearby, sbi, branch]","[sbi, branch]","[sbi, branch]",{},"{branch, sbi}",{}
